<h1><center>Acrobot - RK4</center></h1>

In [1]:
import meshcat
import meshcat.geometry as geom
import meshcat.transformations as tf
import numpy as np
import time
import jax.numpy as jnp
import jax
from jax import jacfwd, hessian
from jaxlie import SE2, SE3
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math

In [2]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
vis = meshcat.Visualizer().open()

center_of_attention = [0, 0, 0]
camera_offset = [2, 0, 0]

translation = tf.translation_matrix(camera_offset)
rotation = tf.rotation_matrix(-np.pi/2, [0, 0, 1])
camera_transform = np.dot(rotation, translation)
vis["/Cameras/default"].set_transform(camera_transform)
vis["/Cameras/default/rotated/<object>"].set_property("position", camera_offset)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
########################################## Parameters ##########################################

dt = 1e-5 #time step
_g = 9.81 #gravity
steps = 1500000 # iterations

### link 1 ###
_l1 = 0.5 #lenght
_m1 = 10 #mass
q1 = 0 #rads
q_dot1 = 0 #rads/s
q_ddot1 = 0 #rads/s**2
T1 = 0 #torque
I1 = 1/3 * _m1 * _l1**2 #Inertia
_d1 = 0.0 #dampening

### link 2 ###
_l2 = 0.5
_m2 = 10
q2 = 0.0
q_dot2 = 0
q_ddot2 = 0
T2 = 0
I2 = 1/3 * _m2 * _l2**2
_d2 = 0.0

#### Using the equations in https://underactuated.csail.mit.edu/acrobot.html

In [5]:
q = np.array([ q1, q2 ])
q_dot = np.array([ q_dot1, q_dot2 ])
q_ddot = np.array([q_ddot1, q_ddot2])
D = np.diag([_d1, _d2])
B = np.eye(2)

def mass_matrix(q):
    m11 = I1 + I2 + _m2 * (_l1 ** 2) + 2 * _m2 * _l1 * (_l2/2) * np.cos(q[1])
    m12 = I2 + _m2 * _l1 * (_l2/2) * np.cos(q[1])
    m21 = I2 + _m2 * _l1 * (_l2/2) * np.cos(q[1])
    m22 = I2
    return np.array([ [m11, m12], [m21, m22] ])
    
def coriolis(q, q_dot):
    c11 = -2 * _m2 * _l1 * (_l2/2) * np.sin(q[1]) * q_dot[1]
    c12 = -_m2 * _l1 * (_l2/2) * np.sin(q[1]) * q_dot[1]
    c21 = _m2 * _l1 * (_l2/2) * np.sin(q[1]) * q_dot[0]
    c22 = 0.0

    return np.array([ [c11, c12], [c21, c22] ])
    
def gravy(q):
    g1 = _m1 * _g * (_l1 / 2) * np.sin(q[0]) + _m2 * _g * (_l1 * np.sin(q[0]) + (_l2 / 2) * np.sin(q[0] + q[1]))
    g2 = _m2 * _g * (_l2 / 2) * np.sin(q[0] + q[1])
    return np.array([-g1, -g2])

In [9]:
################################################ balancing ################################################
q_des = np.array([np.pi/3, 0])
q_dot_des = np.array([0.0, 0.0])

kp = np.array([10000, 10000])
kd = np.array([30, 30])

def balancing(q, q_dot, q_des, q_dot_des):
    e = q_des- q
    e_dot = q_dot_des - q_dot

    T = kp * e + kd * e_dot
    return T

In [7]:
################################################ RK4 ############################################
    
def angular(q, q_dot, T):
    q_ddot = np.linalg.inv(mass_matrix(q)) @ (B @ T - coriolis(q, q_dot) @ q_dot - gravy(q)- D @ q_dot)
    return q_ddot

def f(y, T):
    q = y[:2]
    q_dot = y[2:]
    q_ddot = angular(q, q_dot, T)
    return np.concatenate((q_dot, q_ddot))

def rk4(y, dt, T):
    k1 = f(y, T)
    k2 = f(y + dt * k1/2, T)
    k3 = f(y + dt * k2/2, T)
    k4 = f(y + dt * k3, T)
    return y + (dt/6) * (k1 + 2 * k2 + 2 * k3 + k4)

### Kinetic and Potential Energy 

In [8]:
################################################ For Graphing ############################################
kinetic_list = []
potential_list = []

m = np.array([_m1, _m2])

def kinetic(q, q_dot):
    v1_sq = (_l1 / 2) ** 2 * q_dot[0] ** 2
    v2_sq = (_l1 * q_dot[0]) ** 2 + (_l2 / 2) ** 2 * q_dot[1] ** 2 + 2 * _l1 * (_l2 / 2) * q_dot[0] * q_dot[1] * np.cos(q[0] - q[1])
    K1 = 0.5 * _m1 * v1_sq + 0.5 * I1 * q_dot[0] ** 2
    K2 = 0.5 * _m2 * v2_sq + 0.5 * I2 * q_dot[1] ** 2
    return K1 + K2
 
def potential(q):
    y1 = -(_l1 / 2) * np.cos(q[0])
    y2 = -_l1 * np.cos(q[0]) - (_l2 / 2) * np.cos(q[0] + q[1])
    U1 = _m1 * _g * y1
    U2 = _m2 * _g * y2
    return U1 + U2

def total_energy(q, q_dot):
    return kinetic(q, q_dot) + potential(q_des)

### Simulation

In [10]:
######################### Animation RK4 Method #########################
### pivot 1 ### 
pivot_1 = vis["base_pivot"].set_object(geom.Sphere(0.01))
### link 1 ###
link = vis["pendulum_link"]
link.set_object(geom.Box([0.01, 0.01, -_l1]))

### pivot 2 ###
vis["base_pivot_2"].set_object(geom.Sphere(0.01))
### link 2 ###
link2 = vis["pendulum_link_2"]
link2.set_object(geom.Box([0.01, 0.01, -_l2]))

y = np.array([q1, q2, q_dot1, q_dot2])

for t in range(steps):
                                
    T = balancing(q, q_dot, q_des, q_dot_des)
    q_ddot = angular(q, q_dot, T)
    y = rk4(y, dt, T)
    
    q = y[:2]
    q_dot = y[2:]

    K = kinetic(q, q_dot)
    U = potential(q)

    kinetic_list.append(K)
    potential_list.append(U)

    if t % 50 == 0:
        
        #print(q[0], q[1], q_dot[0], q_dot[1])

        ### link 1 ### 
        r_1 = tf.rotation_matrix(q[0], [0, 1, 0])
        t_1 = tf.translation_matrix([0.0, 0.0, (_l1 / 2)])
    
        vis["pendulum_link"].set_transform(np.dot(r_1, t_1))
    
    ### pivot 2 ###
        pt_2 = tf.translation_matrix([0.0, 0.0, _l1])
        vis["base_pivot_2"].set_transform(np.dot(r_1, tf.translation_matrix([0.0, 0.0, _l1])))

    ### link 2 ###
        t_2 = np.dot(r_1, tf.translation_matrix([0.0, 0.0, _l1]))  # relative to origin
        r_2 = tf.rotation_matrix(q[1], [0.0, 1.0, 0.0])
        t2 = tf.translation_matrix([0.0, 0.0, _l2/2])

        vis["pendulum_link_2"].set_transform(np.dot(t_2, np.dot(r_2, t2)))
        
    time.sleep(dt)

KeyboardInterrupt: 

In [ ]:
time = list(range(steps))

In [ ]:
plt.plot(time, kinetic_list)
SSplt.xlabel("Steps")
plt.ylabel("Kinetic energy (J)")
plt.title("Kinetic energy over steps")

In [ ]:
plt.plot(time, potential_list)
plt.xlabel("Steps")
plt.ylabel("Potential energy (J)")
plt.title("Potential Energy over Steps")

In [ ]:
E = (np.array(kinetic_list) + np.array(potential_list)).tolist()
plt.plot(time, E)
plt.title("Total Energy over Time using RK4");

In [ ]:
threshold = 1e6

for i in range(1, len(E)):
    diff = abs(E[i] - E[i-1])
    if math.isnan(E[i]) or diff > threshold:
        print(f"Anomaly at step {i}: value={E[i]}, diff from previous={diff}")
        break
